In [19]:
import pandas as pd
import numpy as np
import torch
from chronos import BaseChronosPipeline
from sklearn.preprocessing import StandardScaler

import os
from glob import glob
from pathlib import Path
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

# override all pandas display limits
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
# load parquet files
data_path = Path("../../data")
notebooks_path = Path(os.getcwd())
data_dir = {}

##unsafe
# for file_path in list((notebooks_path / data_path).glob("*.parquet")):
#     print(f"Reading {file_path}")
#     # retrieve the name of the file without the extension for all OS
#     data = pd.read_parquet(file_path)
#     # if "Time" in df.columns:
#     #     df["Time"] = pd.to_datetime(df["Time"])
#     data_dir[str(file_path).replace("\\", "/").split("/")[-1].split(".")[0].replace("-", "_")] = data

for file_path in (notebooks_path / data_path).glob("set*.parquet"):
    var_name = file_path.stem.replace("-", "_")
    globals()[var_name] = pd.read_parquet(file_path)
    print(f"Reading {file_path} -> {var_name}")


ID_vars = ["PatientID", "Time", "RecordID"] # TODO 
# stationary variables
stationary_vars = ["Age", "Gender", "Height"] #, "ICUType"]
# dynamic variables
dynamic_vars = set_a.columns.difference(stationary_vars + ID_vars).tolist()

Reading /home/pbaertschi/ICU-TimeSeries-Mortality-Prediction/notebooks/4_Foundation_Models/../../data/set-a-filled.parquet -> set_a_filled
Reading /home/pbaertschi/ICU-TimeSeries-Mortality-Prediction/notebooks/4_Foundation_Models/../../data/set-b.parquet -> set_b
Reading /home/pbaertschi/ICU-TimeSeries-Mortality-Prediction/notebooks/4_Foundation_Models/../../data/set-a.parquet -> set_a
Reading /home/pbaertschi/ICU-TimeSeries-Mortality-Prediction/notebooks/4_Foundation_Models/../../data/set-c-filled.parquet -> set_c_filled
Reading /home/pbaertschi/ICU-TimeSeries-Mortality-Prediction/notebooks/4_Foundation_Models/../../data/set-c.parquet -> set_c
Reading /home/pbaertschi/ICU-TimeSeries-Mortality-Prediction/notebooks/4_Foundation_Models/../../data/set-b-filled.parquet -> set_b_filled


In [6]:
def preprocess_parquet_for_lstm(key, scaler=None, fit_scaler=False):
    labelname = 'In-hospital_death'
    df = globals()[f"set_{key}_filled"].copy()

    # Sort and fill NaNs
    df = df.sort_values(["RecordID", "Time"])
    df[feature_cols] = df[feature_cols].fillna(0)

    # raise NotImplementedError("Encode cathegories")

    # --- Fit scaler on all feature data if requested ---
    if fit_scaler or scaler is None:
        scaler = StandardScaler()
        scaler.fit(df[feature_cols])

    # --- Apply scaling ---
    df[feature_cols] = scaler.transform(df[feature_cols])

    # Group by patient
    X = []
    y = []
    for pid, group in df.groupby("RecordID"):
        group = group.sort_values("Time")
        X.append(group[feature_cols].values)
        y.append(group[labelname].iloc[0])

    X_tensor = torch.tensor(np.stack(X)).float()  # (n_patients, seq_len, n_features)
    y_tensor = torch.tensor(y).float()            # (n_patients,)

    return X_tensor, y_tensor, scaler  # return scaler for reuse on val/test

feature_cols = dynamic_vars + stationary_vars
X_train, y_train, fitted_scaler = preprocess_parquet_for_lstm("a", fit_scaler = True)
# len(preprocess_parquet_for_lstm("b", scaler = fitted_scaler))
X_val, y_val , _    = preprocess_parquet_for_lstm("b", scaler = fitted_scaler)
X_test, y_test , _  = preprocess_parquet_for_lstm("c", scaler = fitted_scaler)

In [14]:
num_patients, sequence_length, num_features  = X_train.size()

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load the pretrained Chronos model
model_id = "amazon/chronos-t5-small"  # Example model; choose the one that fits your needs
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",  # use "amazon/chronos-bolt-small" for the corresponding Chronos-Bolt model
    device_map=device,  # use "cpu" for CPU inference
    torch_dtype=torch.bfloat16,
)


In [36]:
X_tensor = X_train.clone().to(device)
embeddings_list = []


for feature_idx in range(num_features):
    feature_data = X_tensor[:, :, feature_idx].to(pipeline.model.device)

    # Fix: move tokenizer's boundaries to the same device
    # pipeline.tokenizer.boundaries = pipeline.tokenizer.boundaries.to(feature_data.device)

    with torch.no_grad():
        embeddings, _ = pipeline.embed(feature_data)
        embeddings_list.append(embeddings)

# Stack embeddings: Shape will be (num_patients, num_features, embedding_dim)
embeddings_stack = torch.stack(embeddings_list, dim=1)

# Aggregate embeddings across features, e.g., by averaging
final_embeddings = embeddings_stack.mean(dim=1)  # Shape: (num_patients, embedding_dim)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [34]:
feature_data.device


device(type='cuda', index=0)